Connect to Snowflake

In [59]:
import toml
import snowflake.connector
from snowflake.snowpark import Session

config = toml.load("../../.snowflake/connections.toml")
print(config)
sf_config = config["connections"]["test_conn"]
print("test",sf_config)

#sf_config = config["connections.test_conn"]

import snowflake.connector

conn = snowflake.connector.connect(
    user=sf_config["user"],
    password=sf_config["password"],
    account=sf_config["account"],
    warehouse=sf_config["warehouse"],
    database=sf_config["database"],
    schema=sf_config["schema"]
)

connection_parameters = {
        "account": sf_config["account"],
        "user": sf_config["user"],
        "password": sf_config["password"],
        "warehouse": sf_config["warehouse"],
        "database": sf_config["database"],
        "schema": sf_config["schema"]
    }
session = Session.builder.configs(connection_parameters).create()
print("Session ")
print("Snowpark Session Test:", session.get_current_schema())

try:
    cursor = conn.cursor()
    #cursor.execute("USE SCHEMA SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL;")
    #cursor.execute("SELECT * from CUSTOMER LIMIT 10")
    #df = cursor.fetchall()
    #print(df)
except Exception as e:
    print(f"this failed due to {e.args}, {e}")

session.close()


{'LPYEMFW-LLA71191': {'account': 'LPYEMFW-LLA71191', 'user': 'DASDK350', 'authenticator': 'snowflake', 'password': 'sad_KAPEED534', 'database': 'SNOWFLAKE_SAMPLE_DATA', 'schema': 'TPCDS_SF100TCL', 'warehouse': 'COMPUTE_WH'}, 'connections': {'test_conn': {'account': 'LPYEMFW-LLA71191', 'user': 'DASDK350', 'authenticator': 'snowflake', 'password': 'sad_KAPEED534', 'database': 'TASTY_BYTES_SAMPLE_DATA', 'schema': 'PUBLIC', 'warehouse': 'COMPUTE_WH'}}}
test {'account': 'LPYEMFW-LLA71191', 'user': 'DASDK350', 'authenticator': 'snowflake', 'password': 'sad_KAPEED534', 'database': 'TASTY_BYTES_SAMPLE_DATA', 'schema': 'PUBLIC', 'warehouse': 'COMPUTE_WH'}
Session 
Snowpark Session Test: "PUBLIC"


In [64]:

from snowflake.snowpark.functions import sproc, udf, udtf, call_udf, col, lit
from snowflake.snowpark.types import IntegerType, StructType, StructField
from snowflake.snowpark import Session
#from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
connection_parameters = {
        "account": sf_config["account"],
        "user": sf_config["user"],
        "password": sf_config["password"],
        "warehouse": sf_config["warehouse"],
        "database": sf_config["database"],
        "schema": sf_config["schema"]
    }
print(connection_parameters)
session = None
if not session:
    session = Session.builder.configs(connection_parameters).create()

#below doesn;t work as it is in private preview
#session = Session.builder.configs(SnowflakeLoginOptions("test_conn")).create()
session.query_tag = "func-gen"

print(session.get_current_database())
# Execute the query



{'account': 'LPYEMFW-LLA71191', 'user': 'DASDK350', 'password': 'sad_KAPEED534', 'warehouse': 'COMPUTE_WH', 'database': 'TASTY_BYTES_SAMPLE_DATA', 'schema': 'PUBLIC'}
"TASTY_BYTES_SAMPLE_DATA"


Anonymous Stored Procedure (with lambda)

In [65]:
from snowflake.snowpark.functions import sproc
add_one = sproc(
  lambda session, x: session.sql(f"select {x} + 1").collect()[0][0],
  input_types=[IntegerType()], return_type=IntegerType(),
  packages=["snowflake-snowpark-python==1.13.0"])


ret = add_one(1)
print(f"add_one: {ret}")

SnowparkSessionException: (1409): More than one active session is detected. When you call function 'udf' or use decorator '@udf', you must specify the 'session' parameter if you created multiple sessions.Alternatively, you can use 'session.udf.register' to register UDFs

Named stored procedure (with lambda)

In [14]:
add_two = sproc(
  lambda session, x: session.sql(f"select {x} + 2").collect()[0][0],
  name="add_two_proc", replace=True,
  input_types=[IntegerType()], return_type=IntegerType(),
  packages=["snowflake-snowpark-python"])

ret = session.call("add_two_proc", 1)
print(f"add_two: {ret}")

SnowparkSessionException: (1409): More than one active session is detected. When you call function 'udf' or use decorator '@udf', you must specify the 'session' parameter if you created multiple sessions.Alternatively, you can use 'session.udf.register' to register UDFs

Registered stored procedure

In [ ]:
@sproc(
  name="add_three", replace=True,
  is_permanent=True, stage_location="@int_stage",
  packages=["snowflake-snowpark-python"])
def add_three(session: Session, x: int) -> int:
  return session.sql(f"select {x} + 3").collect()[0][0]

# alternative
# session.sproc.register(
#   func=add_three, name="add_three", replace=True,
#   is_permanent=True, stage_location="@int_stage",
#   packages=["snowflake-snowpark-python"])

ret = session.sql("call add_three(1)").collect()[0][0]
print(f"add_three: {ret}")

Anonymous UDF (with lambda)

In [ ]:
add_five = udf(lambda x: x+5,
  input_types=[IntegerType()], return_type=IntegerType())

df = session.create_dataframe([[1]]).to_df("a")
ret = df.select(add_five(col("a"))).collect()[0][0]
print(f"add_five: {ret}")

Named UDF (with lambda)

In [ ]:
add_six = udf(lambda x: x+6,
  name="add_six_proc", replace=True,
  input_types=[IntegerType()], return_type=IntegerType())

ret = session.sql("select add_six_proc(1)").collect()[0][0]
print(f"add_six: {ret}")

Registered UDF

In [ ]:
@udf(
    name="add_seven", replace=True,
    is_permanent=True, stage_location="@int_stage")
def add_seven(x: int) -> int:
    return x+7

df = session.create_dataframe([[1]], schema=["a"])
ret = df.select(call_udf("add_seven", col("a"))).collect()[0][0]
print(f"add_seven: {ret}")

Registered UDTF

In [ ]:
class GetTwo:
  def process(self, n):
    yield(1, )
    yield(n, )

get_two = udtf(GetTwo, 
  output_schema=StructType([StructField("number", IntegerType())]),
  input_types=[IntegerType()])

# SELECT * FROM ( TABLE ("TEST"."PUBLIC".SNOWPARK_TEMP_TABLE_FUNCTION_2PR5R5RI4E(3 :: INT) ))
ret = session.table_function(get_two(lit(3))).collect()
print(f"get_two: {ret}")